### 2-1. 국내 기후 데이터 정제
- 단순하게 채소별로 묶여있는 기후 데이터를 머신러닝, LSTM 모델에 적합한 데이터 형식으로 변경

In [12]:
import pandas as pd
import numpy as np
import os

In [11]:
pd.read_csv('./data/climate_merged/당근_na.csv', index_col = 0)

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,지점,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),...,기온 QC플래그,강수량 QC플래그,풍속 QC플래그,풍향 QC플래그,습도 QC플래그,현지기압 QC플래그,해면기압 QC플래그,일조 QC플래그,일사 QC플래그,지면온도 QC플래그
0,288,2010-01-01 00:00,-8.4,NaN,0.5,360.0,44.0,1.4,-18.5,1020.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,288,2010-01-01 01:00,-8.9,NaN,1.4,290.0,48.0,1.5,-17.9,1020.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,288,2010-01-01 02:00,-9.1,NaN,0.4,0.0,51.0,1.6,-17.4,1020.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,288,2010-01-01 03:00,-9.5,NaN,0.7,290.0,58.0,1.7,-16.2,1021.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,288,2010-01-01 04:00,-8.0,NaN,1.4,340.0,52.0,1.7,-16.1,1020.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6884,188,2021-10-14 20:00,21.4,NaN,1.5,320.0,87.0,22.1,19.1,1016.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,NaN
6885,188,2021-10-14 21:00,21.1,NaN,1.3,320.0,87.0,21.7,18.8,1016.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,NaN
6886,188,2021-10-14 22:00,21.3,NaN,2.0,320.0,81.0,20.4,17.8,1015.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,NaN
6887,188,2021-10-14 23:00,20.9,NaN,2.9,320.0,80.0,19.7,17.3,1015.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,NaN


In [13]:
def max_or_zero(series):
    if series.notna().sum() == 0 :
        return 0
    else :
        return series[series.notna()].max()
    
def min_or_zero(series):
    if series.notna().sum() == 0 :
        return 0
    else :
        return series[series.notna()].min()

def mean_or_zero(series):
    if series.notna().sum() == 0 :
        return 0
    else :
        return series[series.notna()].mean()

In [14]:
convert_datas = {
    'Temp' : lambda x : x.mean(),
    'Rainfall' : max_or_zero,
    'WindSpeed' : max_or_zero,
    'Humidity' : lambda x : x.mean(),
    'SteamPressure' : lambda x : x.mean(),
    'LandAp' : lambda x : x.mean(),
    'SeaAp' : lambda x : x.mean(),
    'Sunshine' : mean_or_zero,
    'Snow' : max_or_zero,
    'LandTemp' : lambda x : x.mean(),
}

In [15]:
def convert_df(date, df) :
    morning = df.loc[date : date + ' 11:59:59']
    afternoon = df.loc[date + ' 12:00:00' : date]
    
    lines = {}
    
    for column, func in convert_datas.items():
        lines['m_' + column] = func(morning[column])
        
    for column, func in convert_datas.items():
        lines['a_' + column] = func(afternoon[column])
    
    lines['TempMin'] = min_or_zero(morning.Temp)
    lines['TempMax'] = max_or_zero(afternoon.Temp)
    lines['Freeze'] = (morning.Temp < morning.Dewpoint).sum() + (afternoon.Temp < afternoon.Dewpoint).sum()
    return lines

def concat_horizontal(df) : 
    location_list = df.Location.unique()
    merge_df_list = []
    for location in location_list :
        print(location, 'start')
        temp_df = df[df.Location == location].sort_values('Date')
        temp_list = []
        for date in dates :
            temp_list.append(pd.DataFrame(convert_df(date, temp_df), index = [date]))
        temp_df = pd.concat(temp_list)
        temp_df = temp_df.add_suffix(f'_{location}')
        merge_df_list.append(temp_df.ffill())
    return pd.concat(merge_df_list, axis = 1)

In [16]:
date_list = list(pd.date_range('2010-01-01', '2021-10-15'))
dates = []
for date in date_list:
    dates.append(str(date).split(' ')[0])

In [17]:
columns = ['지점', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)','해면기압(hPa)',
       '일조(hr)', '적설(cm)', '지면온도(°C)']

columns_rename = [
    'Location', 'Date', 'Temp', 'Rainfall', 'WindSpeed', 'Humidity', 'SteamPressure', 'Dewpoint', 'LandAp', 'SeaAp',
    'Sunshine', 'Snow', 'LandTemp']

In [51]:
path_lists = os.listdir('./data/climate_merged')
for path in path_lists:
    if 'for' in path :
        continue
    print(path, 'start')
    df = pd.read_csv(f'./data/climate_merged/{path}')
    df = df[columns]
    df.columns = columns_rename

    df['Date'] = pd.to_datetime(df.Date)
    df.set_index('Date', inplace = True)
    
    temp = concat_horizontal(df)
    print(temp.isnull().sum().sum())
    temp.to_csv(f'./data/climate_final/{path}', encoding = 'cp949')

고추_na.csv start


C:\src\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


136 start
221 start
245 start
271 start
0
당근_na.csv start


C:\src\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


288 start
188 start
0
마늘_na.csv start


...

251 start
156 start
100 start
189 start
129 start
172 start
177 start
216 start
217 start
261 start
268 start
95727
양파_na.csv start


C:\src\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


165 start
278 start
288 start
0
토마토_na.csv start


C:\src\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


156 start
258 start
159 start
236 start
101 start
456


In [9]:
pd.read_csv('./data/climate_final/국내/당근_na.csv', index_col = 0)

,m_Temp_288,m_Rainfall_288,m_WindSpeed_288,m_Humidity_288,m_SteamPressure_288,m_LandAp_288,m_SeaAp_288,m_Sunshine_288,m_Snow_288,m_LandTemp_288,...,a_Humidity_188,a_SteamPressure_188,a_LandAp_188,a_SeaAp_188,a_Sunshine_188,a_Snow_188,a_LandTemp_188,TempMin_188,TempMax_188,Freeze_188
2010-01-01,-7.591667,0.0,1.4,53.083333,1.800000,1021.008333,1022.408333,0.675,0.0,-6.891667,...,55.750000,4.300000,1017.691667,1019.991667,0.828571,0.0,7.333333,-0.9,6.8,0
2010-01-02,-5.391667,0.0,2.0,67.083333,2.733333,1018.166667,1019.566667,0.000,0.0,-4.416667,...,55.583333,6.841667,1013.708333,1015.941667,0.785714,0.0,10.300000,-0.1,14.2,0
2010-01-03,-2.091667,0.0,2.8,51.833333,2.666667,1017.925000,1019.316667,0.675,0.0,-4.091667,...,54.333333,4.016667,1017.291667,1019.591667,0.871429,0.0,6.008333,1.9,5.2,0
2010-01-04,-2.000000,0.0,1.2,43.583333,2.300000,1014.450000,1015.841667,0.000,0.0,-2.391667,...,59.500000,6.075000,1010.800000,1013.058333,0.328571,0.0,7.283333,0.0,13.8,0
2010-01-05,-3.108333,0.0,4.0,61.000000,2.908333,1016.458333,1017.858333,0.650,0.0,-3.908333,...,67.333333,4.408333,1019.833333,1022.133333,0.000000,0.0,2.066667,0.3,1.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-11,21.483333,5.4,1.7,91.500000,23.491667,1015.983333,1016.983333,0.000,0.0,21.800000,...,79.250000,21.483333,1015.475000,1017.850000,0.425000,0.0,24.533333,21.4,25.7,0
2021-10-12,17.616667,2.9,2.3,83.666667,16.775000,1021.750000,1022.750000,0.000,0.0,17.733333,...,79.416667,22.733333,1018.958333,1021.325000,0.000000,0.0,22.500000,23.1,23.6,0
2021-10-13,18.375000,0.0,1.8,73.750000,15.550000,1023.850000,1024.850000,0.360,0.0,18.458333,...,81.333333,23.458333,1018.958333,1021.316667,0.000000,0.0,22.958333,23.1,24.0,0
2021-10-14,16.766667,0.0,0.9,89.166667,16.841667,1020.691667,1021.691667,0.400,0.0,18.233333,...,73.333333,21.333333,1016.025000,1018.366667,0.957143,0.0,25.383333,23.6,26.6,0
